 **(Batch) Gradient Descent**:


In [ ]:

def gradient():
    X = data_input
    Y = labels
    m = X.shape[1]  # Number of training examples
    parameters = initialize_parameters(layers_dims)
    for i in range(0, num_iterations):
    # Forward propagation
        a, caches = forward_propagation(X, parameters)
    # Compute cost
        cost_total = compute_cost(a, Y)  # Cost for m training examples
    # Backward propagation
        grads = backward_propagation(a, caches, parameters)
    # Update parameters
        parameters = update_parameters(parameters, grads)
    # Compute average cost
        cost_avg = cost_total / m
        



- **Stochastic Gradient Descent**:


In [2]:
def stochastic():
    X = data_input
    Y = labels
    m = X.shape[1]  # Number of training examples
    parameters = initialize_parameters(layers_dims)
    for i in range(0, num_iterations):
        cost_total = 0
        for j in range(0, m):
        # Forward propagation
            a, caches = forward_propagation(X[:,j], parameters)
        # Compute cost
            cost_total += compute_cost(a, Y[:,j])  # Cost for one training example
        # Backward propagation
            grads = backward_propagation(a, caches, parameters)
        # Update parameters
            parameters = update_parameters(parameters, grads)
    # Compute average cost
        cost_avg = cost_total / m


**mini batch gradient descent**

In [3]:
# GRADED FUNCTION: random_mini_batches

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):

    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1, m))
    
    inc = mini_batch_size

    num_complete_minibatches = math.floor(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):

        mini_batch_X =  shuffled_X[:, k*mini_batch_size :(k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size :(k+1)*mini_batch_size]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:

        mini_batch_X = shuffled_X[:,(mini_batch_size*num_complete_minibatches):m ]
        mini_batch_Y = shuffled_Y[:,(mini_batch_size*num_complete_minibatches):m ]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

### update_parameters_with_momentum



$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}\tag{3}$$

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}\tag{4}$$

In [ ]:
# GRADED FUNCTION: update_parameters_with_momentum

def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):


    L = len(parameters) // 2 # number of layers in the neural networks
    
    for l in range(1, L + 1):
    
        v["dW" + str(l)] = beta*v["dW" + str(l)]+(1-beta)*(grads['dW' + str(l)])
        v["db" + str(l)] = beta*v["db" + str(l)]+(1-beta)*(grads['db' + str(l)])
        parameters["W" + str(l)] =parameters["W" + str(l)]-learning_rate*v["dW" + str(l)]
        parameters["b" + str(l)] =parameters["b" + str(l)]-learning_rate*v["db" + str(l)]
        
        
    return parameters, v

### update_parameters_with_adam

Now, implement the parameters update with Adam. Recall the general update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$



In [ ]:
# GRADED FUNCTION: update_parameters_with_adam

def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    for l in range(1, L + 1):
 
        v["dW" + str(l)] = beta1*v["dW" + str(l)]+(1-beta1)*(grads['dW' + str(l)])
        v["db" + str(l)] = beta1*v["db" + str(l)]+(1-beta1)*(grads['db' + str(l)])   

        v_corrected["dW" + str(l)] =v["dW" + str(l)]/(1 - math.pow(beta1,t))
        v_corrected["db" + str(l)] =v["db" + str(l)]/(1 - math.pow(beta1,t))

         
        s["dW" + str(l)] = beta2*s["dW" + str(l)] +(1-beta2)*(grads['dW' + str(l)])**2
        s["db" + str(l)] = beta2*s["db" + str(l)] +(1-beta2)*(grads['db' + str(l)])**2

        
        s_corrected["dW" + str(l)] =s["dW" + str(l)]/(1 - math.pow(beta2,t))
        s_corrected["db" + str(l)] =s["db" + str(l)]/(1 - math.pow(beta2,t))
        

        parameters["W" + str(l)] =parameters["W" + str(l)] - learning_rate* v_corrected["dW" + str(l)]/(np.sqrt(s_corrected["dW" + str(l)])+epsilon)
        parameters["b" + str(l)] =parameters["b" + str(l)] - learning_rate* v_corrected["db" + str(l)]/(np.sqrt(s_corrected["db" + str(l)])+epsilon)


    return parameters, v, s, v_corrected, s_corrected

### Learning Rate Decay and Scheduling

$$\alpha = \frac{1}{1 + decayRate \times epochNumber} \alpha_{0}$$


$$\alpha = \frac{1}{1 + decayRate \times \lfloor\frac{epochNum}{timeInterval}\rfloor} \alpha_{0}$$


In [ ]:
def update_learning_rate(learning_rate0, epoch_num, decay_rate):
 
    learning_rate=learning_rate0/(1+epoch_num*decay_rate)
    
    return learning_rate